In [1]:
import numpy as np
import uDMA
import Controller
from pynq import xlnk
from pynq import overlay
import socket
import sys
import math
import time

In [2]:
#Control gpio address
TriBRAM_Control_Addr              = 0x43C10000
CLK_Control_Addr                  = 0x43C30000
TDC_Control_Addr                  = 0x43C40000
DINT_Control_Addr                 = 0x43C20000
UDMA_Control_Addr                 = 0x43C50000
CMP_Delay_Addr                    = 0x43C00000
Data_length                       = 65536*3#= 32768#the data length for u64, 8bytes
UDP_MAX                           = 32768#the max byte length for UDP
UDP_IP                            = "192.168.0.186"
UDP_PORT                          = 8001
TARGET                            = (UDP_IP,UDP_PORT)
fs                                = 125e6#Sampling frequency
             
#Control paramters
REF_Clk_parameter                 = 0x0000000A
Trigger_stop_parameter            = int(1200e-6*fs)#= 0x00001D4C#60us = 0x00003A98#120us #= 0x00007530#240us 
Trigger_start_parameter           = 0x00000006
DINT_Fall_parameter               = int(620e-6*fs)#= 0x00001D4C#60us = 0x00003A98#120us= 0x00007530#240us
DINT_Keep_parameter               = DINT_Fall_parameter-10#
Window_start_parameter            = 0x00000038#22clks#0x00000001#0clks#0x00000038#57clks
Window_stop_parameter             = Trigger_stop_parameter + Window_start_parameter -2#22clks
Count_parameter                   = Window_stop_parameter*2 #+ 0x00000006 + 0x00001D4C#60us #0x00003ADF#120us
BC_Interval_addr_paramter         = 0x00000100#0x00001000 corresponds addr 1 in bram excha
CMPs_Delay_Para                   = 0x00000002

single_type_INT                   = 0x00000002
double_type_INT                   = 0x00000000

Current_threshold                 = 10000#290mA
MESSAGE = b'uDMA start!'

In [3]:
print(Trigger_stop_parameter)

150000


In [4]:
bitstream = overlay.Overlay('system_wrapper.bit')
bitstream.download()

/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


In [5]:
allo = xlnk.Xlnk()
data = allo.cma_array([math.floor(Data_length*8/UDP_MAX),UDP_MAX],dtype=np.uint8)

In [6]:
ctrl = Controller.Controller(TriBRAM_Control_Addr, CLK_Control_Addr, TDC_Control_Addr, DINT_Control_Addr, CMP_Delay_Addr)
udma = uDMA.uDMA(UDMA_Control_Addr,data.physical_address)
udma.set_memaddr(data.physical_address)
udma.set_length(Data_length,8)
ctrl.system_rst_on()
# Settings for the trigger and windows signals and the initial values, the defaults are initialed in the 
ctrl.Set_Trigger(
    Count_parameter,
    Trigger_start_parameter,
    Trigger_stop_parameter,
    Window_start_parameter,
    Window_stop_parameter,
    DINT_Fall_parameter,
    DINT_Keep_parameter
)
ctrl.Set_BM(BC_Interval_addr_paramter, Current_threshold)
ctrl.system_rst_off()
udma.reset_off()
#If the locking period or reference clock changes, this paramter should be modified
ctrl.Set_Delay(CMPs_Delay_Para)#defalut one is 2
ctrl.Set_Offset(3000)#defalut value is 0x7D0
print ("UDP target IP:", UDP_IP)
print ("UDP target port", UDP_PORT)
print ("message:", MESSAGE)
sock = socket.socket(
    socket.AF_INET,
    socket.SOCK_DGRAM)

UDP target IP: 192.168.0.186
UDP target port 8001
message: b'uDMA start!'


In [1684]:
ctrl.system_rst_on()
ctrl.system_rst_off()

In [7]:
ctrl.Set_Offset(3000)

In [8]:
ctrl.Set_Ref(5)#Set the reference clock

In [963]:
ctrl.Active_Control_on(200)

In [967]:
ctrl.Active_Control_off()#Default option

In [1222]:
ctrl.Set_INT_coe(50,50)#Set the INT coe for the feedback part

In [1680]:
ctrl.Iterating(single_type_INT)#Iteration mode, default is the single one

In [2044]:
ctrl.Iterating(double_type_INT)

In [2046]:
# udma.start_off()
udma.start_on()

while(not udma.ifDone()):
    pass
udma.start_off()


# print(udma.ifDone())

data[0,7] = 0xAA
data[(math.floor(Data_length*8/UDP_MAX)-1),7] = 0x55
for i in range(math.floor(Data_length*8/UDP_MAX)):
    sock.sendto(data[i,:],(UDP_IP,UDP_PORT))
    time.sleep(0.07)
print(i)

47


In [149]:
for i in range(math.floor(Data_length*8/UDP_MAX)):
    sock.sendto(data[i,:],TARGET)
    time.sleep(0.07)
print(i)

47


In [10]:
sock.close()